# 搭建ResNet

In [1]:
from torch import nn
import torch
from torch.nn import functional as F

/data/wangyidan-slurm/anaconda3/envs/pt4dm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ResidualBlock(nn.Module):
    '''
    子module
    '''
    
    def __init__(self,inchannel,outchannel,stride=1,shortcut=None):
        super(ResidualBlock,self).__init__()
        
        self.left = nn.Sequential(
            nn.Conv2d(in_channels=inchannel,out_channels=outchannel,
                      kernel_size=3,stride=stride,
                      padding=1,bias=False),
            nn.BatchNorm2d(num_features=outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=outchannel,out_channels=outchannel,
                     kernel_size=3,stride=1,
                     padding=1,bias=False),
            nn.BatchNorm2d(num_features=outchannel)
        )
        
        self.right = shortcut
    
    def forward(self,x):
        '''
        x：size[batch_size,inchannel,h,w]
        '''
        out = self.left(x)
        residual = x if self.right is None else self.right(x)
        
        out += residual
        
        return F.relu(out)

In [3]:
class ResNet(nn.Module):
    '''
    主网络
    包含多个layer，每个layer包含多个residual block
    '''
    
    def __init__(self, num_classes=1000):
        super(ResNet,self).__init__()
        
        self.pre = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=64,
                     kernel_size=7,stride=2,
                     padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        )
        
        # layer
        self.layer1 = self._make_layer(64,64,3)
        self.layer2 = self._make_layer(64,128,4,stride=2)
        self.layer3 = self._make_layer(128,256,6,stride=2)
        self.layer4 = self._make_layer(256,512,3,stride=2)
        
        self.fc = nn.Linear(512,num_classes)
        
    def _make_layer(self,inchannel,outchannel,block_num,stride=1):
        shortcut = nn.Sequential(
            nn.Conv2d(in_channels=inchannel,out_channels=outchannel,kernel_size=1,stride=stride,bias=False),
            nn.BatchNorm2d(outchannel)
        )
        
        layers = []
        layers.append(ResidualBlock(inchannel,outchannel,stride,shortcut))
        
        for i in range(1,block_num):
            layers.append(ResidualBlock(outchannel,outchannel))
        
        return nn.Sequential(*layers) # 都要打包成Sequential
    
    def forward(self,x):
        x = self.pre(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = F.avg_pool2d(x,7)
        x = x.view(x.size(0),-1)
        
        return self.fc(x)

In [4]:
model = ResNet()
input = torch.randn(1,3,224,224)
o = model(input)

### ResNet网络的结构
- Conv2d 会改变图片的形状和特征数量（通道数）
- MaxPool 会改变图片的形状

？ Conv网络中，形状具体是怎么变化的

In [6]:
from torchinfo import summary

In [10]:
summary(model,input_data=input,col_names=["input_size","output_size", "num_params"],
        depth=4,row_settings=["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #
ResNet (ResNet)                          [1, 3, 224, 224]          [1, 1000]                 --
├─Sequential (pre)                       [1, 3, 224, 224]          [1, 64, 56, 56]           --
│    └─Conv2d (0)                        [1, 3, 224, 224]          [1, 64, 112, 112]         9,408
│    └─BatchNorm2d (1)                   [1, 64, 112, 112]         [1, 64, 112, 112]         128
│    └─ReLU (2)                          [1, 64, 112, 112]         [1, 64, 112, 112]         --
│    └─MaxPool2d (3)                     [1, 64, 112, 112]         [1, 64, 56, 56]           --
├─Sequential (layer1)                    [1, 64, 56, 56]           [1, 64, 56, 56]           --
│    └─ResidualBlock (0)                 [1, 64, 56, 56]           [1, 64, 56, 56]           --
│    │    └─Sequential (left)            [1, 64, 56, 56]           [1, 64, 56, 56]           --
│    │    │    └─Conv2d (0)    